In [68]:
!pip install python-dotenv


In [81]:
#TASK 1

!pip install groq


import os
from dotenv import load_dotenv
from groq import Groq

load_dotenv()

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

In [82]:
conversation1 = [
    {"role": "user", "content": "Hi, how are you?"},
    {"role": "assistant", "content": "I’m good! How about you?"},
    {"role": "user", "content": "I’m learning AI, it’s interesting!"},
    {"role": "assistant", "content": "That’s great! Which part of AI interests you?"}
]

print("Conversation 1:")
for msg in conversation1:
    print(f"{msg['role']}: {msg['content']}")


Conversation 1:
user: Hi, how are you?
assistant: I’m good! How about you?
user: I’m learning AI, it’s interesting!
assistant: That’s great! Which part of AI interests you?


In [83]:
def get_last_n_turns(conv, n=2):
    return conv[-n:]

last2 = get_last_n_turns(conversation1, n=2)
print("\nLast 2 turns:")
print(last2)



Last 2 turns:
[{'role': 'user', 'content': 'I’m learning AI, it’s interesting!'}, {'role': 'assistant', 'content': 'That’s great! Which part of AI interests you?'}]


In [84]:
def truncate_message(msg, char_limit=50):
    return {"role": msg["role"], "content": msg["content"][:char_limit]}

truncated_last = [truncate_message(msg, char_limit=50) for msg in last2]
print("\nTruncated last 2 turns by 50 characters:")
print(truncated_last)



Truncated last 2 turns by 50 characters:
[{'role': 'user', 'content': 'I’m learning AI, it’s interesting!'}, {'role': 'assistant', 'content': 'That’s great! Which part of AI interests you?'}]


In [85]:
def summarize_conversation(conv):
    text = ""
    for msg in conv:
        text += f"{msg['role']}: {msg['content']}\n"

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "Summarize the following conversation clearly in one or two sentences."},
            {"role": "user", "content": text},
        ],
    )

    summary_text = response.choices[0].message.content.strip()

    summarized_conv = []
    for line in summary_text.split("\n"):
        if ":" in line:
            role, content = line.split(":", 1)
            summarized_conv.append({"role": role.strip(), "content": content.strip()})

    if not summarized_conv:
        summarized_conv = [{"role": "assistant", "content": summary_text}]

    return summarized_conv


In [86]:
def periodic_summarization(conv, k, run_number):
    """
    Summarize and replace conversation after every k-th run.
    """
    if run_number % k == 0:
        summarized = summarize_conversation(conv)
        return summarized
    return conv


In [87]:
conversation2 = [
    {"role": "user", "content": "Do you like movies?"},
    {"role": "assistant", "content": "Yes, I love movies! What genre do you enjoy?"},
    {"role": "user", "content": "I like science fiction, it’s so imaginative."},
    {"role": "assistant", "content": "Great choice! Sci-fi really pushes creativity."}
]

run_number = 2
conversation2 = periodic_summarization(conversation2, k=2, run_number=run_number)
print("\nPeriodic summarization for conversation2 (after 2nd run):")
print(conversation2)



Periodic summarization for conversation2 (after 2nd run):
[{'role': 'assistant', 'content': "A user and an assistant had a conversation about movies, discussing the user's preference for science fiction films due to their imaginative nature. The assistant agreed, appreciating the creative aspect of science fiction."}]


In [88]:
last2 = get_last_n_turns(conversation2, n=2)
print("\nLast 2 turns:")
print(last2)

truncated_50 = [truncate_message(msg, char_limit=50) for msg in last2]
print("\nTruncated by 50 characters:")
print(truncated_50)

truncated_30 = [truncate_message(msg, char_limit=30) for msg in last2]
print("\nTruncated by 30 characters:")
print(truncated_30)



Last 2 turns:
[{'role': 'assistant', 'content': "A user and an assistant had a conversation about movies, discussing the user's preference for science fiction films due to their imaginative nature. The assistant agreed, appreciating the creative aspect of science fiction."}]

Truncated by 50 characters:
[{'role': 'assistant', 'content': 'A user and an assistant had a conversation about m'}]

Truncated by 30 characters:
[{'role': 'assistant', 'content': 'A user and an assistant had a '}]


In [89]:
#TASK 2

info_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "Full name of the user"},
        "email": {"type": "string", "format": "email", "description": "Email address of the user"},
        "phone": {"type": "string", "description": "Phone number with country code if available"},
        "location": {"type": "string", "description": "City or location of the user"},
        "age": {"type": "integer", "description": "Age of the user in years"}
    },
    "required": ["name", "email", "phone", "location", "age"]
}


In [90]:
sample_chats = [
    "Hi, my name is Virat. You can reach me at virat@example.com or +91-9876543210. I live in Bangalore and I am 22 years old.",
    "Hello, I'm Priya Sharma. Email: priya.sharma@email.com, Phone: 555-234-5678. Location: Mumbai. Age: 25.",
    "Hey, this is Rahul. My contact rahul123@gmail.com, phone number 987-123-4567. I am 28 years old and stay in Delhi."
]


In [91]:
import os
from dotenv import load_dotenv
from groq import Groq
import json

load_dotenv()

client = Groq(api_key=os.getenv("GROQ_API_KEY"))


info_schema = {
    "name": "string",
    "email": "string",
    "phone": "string",
    "location": "string",
    "age": "integer"
}


sample_chats = [
    "Hi, I am Virat. You can reach me at virat@example.com or call me at +91-9876543210. I live in Bangalore and I am 22 years old.",
    "Hello, my name is Priya. My email is priya@example.com, phone number is +91-9123456789. I’m from Delhi and I’m 25."
]

extracted_data = []

for chat in sample_chats:
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an information extractor. "
                    "Extract user details (name, email, phone, location, age) from the conversation. "
                    "Return ONLY valid JSON, matching this structure exactly: "
                    '{"name": string, "email": string, "phone": string, "location": string, "age": integer}. '
                    "Do not include explanations, markdown, or schema. Output only one JSON object."
                )
            },
            {"role": "user", "content": chat},
        ],
    )

    message_obj = response.choices[0].message


    func_call = getattr(message_obj, "function_call", None)
    if func_call and hasattr(func_call, "arguments"):
        func_args = func_call.arguments
    else:
        func_args = message_obj.content

    try:
        parsed_args = json.loads(func_args)
        extracted_data.append(parsed_args)
    except json.JSONDecodeError:
        extracted_data.append({"error": "Invalid JSON returned", "raw": func_args})


for i, data in enumerate(extracted_data):
    print(f"Chat {i+1} extracted info:")
    print(data)
    print("-" * 50)


Chat 1 extracted info:
{'name': 'Virat', 'email': 'virat@example.com', 'phone': '+91-9876543210', 'location': 'Bangalore', 'age': 22}
--------------------------------------------------
Chat 2 extracted info:
{'name': 'Priya', 'email': 'priya@example.com', 'phone': '+91-9123456789', 'location': 'Delhi', 'age': 25}
--------------------------------------------------


In [92]:
from jsonschema import validate, ValidationError

for i, data in enumerate(extracted_data):
    try:
        validate(instance=data, schema=info_schema)
        print(f"Chat {i+1} ✅ Valid data")
    except ValidationError as e:
        print(f"Chat {i+1} ❌ Invalid data: {e}")


Chat 1 ✅ Valid data
Chat 2 ✅ Valid data
